In [1]:
import os
import pandas as pd
import numpy as np

import tensorflow as tf

In [2]:
train = pd.read_csv('data/train_features.csv')
train_labels = pd.read_csv('data/train_labels.csv')
test = pd.read_csv('data/test_features.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [3]:
# id랑 time label을 없앰
X=tf.reshape(np.array(train.iloc[:,2:]),[-1, 600, 6])
X.shape

TensorShape([3125, 600, 6])

In [4]:
# label을 one-hot encoding (61개의 unique 한 label 존재)
y = tf.keras.utils.to_categorical(train_labels['label']) 
y.shape

(3125, 61)

In [5]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Activation, GRU
from keras import optimizers

def deep_gru():
    model = Sequential()
    model.add(GRU(32, input_shape = (600,6), return_sequences = True))
    model.add(GRU(64, return_sequences = True))
    model.add(GRU(64, return_sequences = True))   
    model.add(GRU(128, return_sequences = False))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(61, activation='softmax'))
    
    #adam = optimizers.adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
    return model

In [7]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [8]:
model1 = deep_gru()
model1.fit(X,y, epochs=100, batch_size=32, verbose = 2, validation_split=0.2, callbacks = [es])

Epoch 1/100
79/79 - 189s - loss: 2.9143 - accuracy: 0.4716 - val_loss: 2.7179 - val_accuracy: 0.4976
Epoch 2/100
79/79 - 161s - loss: 2.6603 - accuracy: 0.4840 - val_loss: 2.6095 - val_accuracy: 0.5040
Epoch 3/100
79/79 - 163s - loss: 2.4821 - accuracy: 0.4928 - val_loss: 2.5749 - val_accuracy: 0.5024
Epoch 4/100
79/79 - 160s - loss: 2.2911 - accuracy: 0.5040 - val_loss: 2.4295 - val_accuracy: 0.5136
Epoch 5/100
79/79 - 157s - loss: 2.0991 - accuracy: 0.5300 - val_loss: 2.2177 - val_accuracy: 0.5232
Epoch 6/100
79/79 - 163s - loss: 1.9352 - accuracy: 0.5436 - val_loss: 2.1490 - val_accuracy: 0.5232
Epoch 7/100
79/79 - 143s - loss: 1.7796 - accuracy: 0.5600 - val_loss: 2.0914 - val_accuracy: 0.5424
Epoch 8/100
79/79 - 145s - loss: 1.6177 - accuracy: 0.5808 - val_loss: 1.9721 - val_accuracy: 0.5424
Epoch 9/100
79/79 - 146s - loss: 1.4753 - accuracy: 0.5964 - val_loss: 1.8535 - val_accuracy: 0.5616
Epoch 10/100
79/79 - 149s - loss: 1.3444 - accuracy: 0.6232 - val_loss: 1.9548 - val_accura

In [9]:
# # model저장
# from keras.callbacks import ModelCheckpoint
# mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

In [10]:
test_X=tf.reshape(np.array(test.iloc[:,2:]),[-1, 600, 6])
test_X.shape

TensorShape([782, 600, 6])

In [12]:
prediction=model1.predict(test_X)

In [13]:
submission.iloc[:,1:]=prediction

In [14]:
submission.to_csv('deepGRU_es_submission.csv', index=False)